In [1]:
import pandas as pd
import time
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb

path = './input/'

start_time = time.time()

train = pd.read_csv(path+"ct-sample_train_10000.csv")
train.columns = ['timestamp', 'sdk', 'platform', 'pub_idx', 'pub_app', 'pub_tagid', 'country', 'shop_idx', 'campaign_group_idx', 'campaign_idx', 'ads_idx', 'format_idx', 'ads_width', 'ads_height', 'adsize', 'bid', 'win', 'click', 'ctr']
test = pd.read_csv(path+"ct-sample_test.csv")

print('[{}] Finished to load data'.format(time.time() - start_time))

[0.0238618850708] Finished to load data


/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
del train['sdk']
del train['platform']
del train['pub_app']
del train['country']
del train['adsize']
del train['pub_tagid']

del test['sdk']
del test['platform']
del test['pub_app']
del test['country']
del test['adsize']
del test['pub_tagid']

In [3]:
sub = pd.DataFrame()

sub['ads_idx'] = test['ads_idx']
sub['ctr'] = test['ctr']
# test.drop('ctr', axis=1, inplace=True)

print('[{}] Start XGBoost Training'.format(time.time() - start_time))

[0.0544619560242] Start XGBoost Training


In [4]:
y = train['ctr']


x1, x2, y1, y2 = train_test_split(train, y, test_size=0.1, random_state=99)

print('[{}] Start XGBoost Training'.format(time.time() - start_time))

[0.0662529468536] Start XGBoost Training


In [5]:
watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]

print watchlist

[(<xgboost.core.DMatrix object at 0x7f6ce0480990>, 'train'), (<xgboost.core.DMatrix object at 0x7f6ce0480850>, 'valid')]


In [6]:
params = {'eta': 0.1, 
          'max_depth': 4, 
          'subsample': 0.9, 
          'colsample_bytree': 0.7, 
          'colsample_bylevel':0.7,
          'min_child_weight':100,
          'alpha':4,
          'objective': 'reg:linear', 
          'eval_metric': 'auc', 
          'random_state': 99, 
          'silent': True}

model = xgb.train(params, xgb.DMatrix(x1, y1), 260, watchlist, maximize=True, verbose_eval=10)

print('[{}] Start XGBoost Training'.format(time.time() - start_time))

[0]	train-auc:1.4289	valid-auc:1.47819
[10]	train-auc:1.43256	valid-auc:1.4818
[20]	train-auc:1.43165	valid-auc:1.48434
[30]	train-auc:1.39941	valid-auc:1.47526
[40]	train-auc:1.39633	valid-auc:1.46601
[50]	train-auc:1.36618	valid-auc:1.42617
[60]	train-auc:1.3744	valid-auc:1.42881
[70]	train-auc:1.36312	valid-auc:1.41064
[80]	train-auc:1.36082	valid-auc:1.41194
[90]	train-auc:1.36305	valid-auc:1.4145
[100]	train-auc:1.35754	valid-auc:1.41096
[110]	train-auc:1.36426	valid-auc:1.4249
[120]	train-auc:1.36196	valid-auc:1.42301
[130]	train-auc:1.36991	valid-auc:1.43031
[140]	train-auc:1.36739	valid-auc:1.42599
[150]	train-auc:1.3659	valid-auc:1.424
[160]	train-auc:1.37032	valid-auc:1.43034
[170]	train-auc:1.3701	valid-auc:1.42714
[180]	train-auc:1.37317	valid-auc:1.43052
[190]	train-auc:1.38006	valid-auc:1.4311
[200]	train-auc:1.38216	valid-auc:1.43113
[210]	train-auc:1.3861	valid-auc:1.43186
[220]	train-auc:1.39072	valid-auc:1.43294
[230]	train-auc:1.38336	valid-auc:1.43101
[240]	train-au

In [7]:
test

,timestamp,pub_idx,shop_idx,campaign_group_idx,campaign_idx,ads_idx,format_idx,ads_width,ads_height,bid,win,click,ctr
0,1520906400,135,95,735,3434,57222,1,320,50,1,0,0,0.0
1,1520906400,135,95,735,3434,57226,1,320,50,1,1,0,0.0
2,1520906400,135,170,731,3418,56376,1,320,50,1,0,0,0.0
3,1520906400,135,327,733,3427,57071,1,320,50,1,1,0,0.0
4,1520906400,135,327,733,3427,57091,1,320,50,1,1,0,0.0
5,1520906400,135,95,734,3432,57249,14,0,0,1,1,0,0.0
6,1520906400,135,170,731,3418,56375,14,0,0,2,1,0,0.0
7,1520906400,135,170,731,3418,56379,14,0,0,1,0,0,0.0
8,1520906400,135,170,731,3418,56383,14,0,0,1,0,0,0.0
9,1520906400,135,278,732,3421,57174,14,0,0,1,1,0,0.0


In [8]:
sub['ctr'] = model.predict(xgb.DMatrix(test), ntree_limit=model.best_ntree_limit)
sub.to_csv('./output/ct-sample_result.csv',index=False)

In [9]:
sub

,ads_idx,ctr
0,57222,0.315339
1,57226,0.234709
2,56376,0.336445
3,57071,0.529027
4,57091,0.529027
5,57249,0.717102
6,56375,0.190851
7,56379,0.343842
8,56383,0.343842
9,57174,0.750116
